## Import Competition Data

In [1]:
import numpy as np
import pandas as pd

property_df  = pd.read_csv("data/properties_2017.csv",low_memory = False)
train_df = pd.read_csv("data/train_2017.csv", parse_dates=["transactiondate"])

In [2]:
property_df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [3]:
train_df.head()

,parcelid,logerror,transactiondate
0,14297519,0.025595,2017-01-01
1,17052889,0.055619,2017-01-01
2,14186244,0.005383,2017-01-01
3,12177905,-0.103410,2017-01-01
4,10887214,0.006940,2017-01-01


## Data Cleaning

In [4]:
# find columns with missing values
missing_percent = property_df.isnull().sum() * 100 / len(property_df)
missing_values_df = pd.DataFrame({'column_name': property_df.columns, 'percent_missing': missing_percent})
print(missing_values_df[missing_values_df.percent_missing > 0].sort_values('percent_missing', ascending=False))

                                               column_name  percent_missing
storytypeid                                    storytypeid        99.945632
basementsqft                                  basementsqft        99.945498
yardbuildingsqft26                      yardbuildingsqft26        99.911363
fireplaceflag                                fireplaceflag        99.827048
architecturalstyletypeid          architecturalstyletypeid        99.796966
typeconstructiontypeid              typeconstructiontypeid        99.774020
finishedsquarefeet13                  finishedsquarefeet13        99.743034
buildingclasstypeid                    buildingclasstypeid        99.573532
pooltypeid10                                  pooltypeid10        99.430326
decktypeid                                      decktypeid        99.417831
finishedsquarefeet6                    finishedsquarefeet6        99.280387
poolsizesum                                    poolsizesum        99.063452
pooltypeid2 

In [5]:
# remove features with more than 70% null values
threshold = 80
null_cols = missing_values_df[missing_values_df.percent_missing > threshold].column_name.tolist()

In [6]:
property_df = property_df.drop(null_cols, axis=1)

In [7]:
property_df.shape

(2985217, 34)

In [8]:
# impute values for geographic categorical features - use mode of broader geographic feature by grouping
# using mode across the feature itself would provide geographical values that don't make sense 
# eg: find most common zip code in the county to impute missing zip

def impute_geographical_feature(df, target_feature, group_feature):
    # mode of target feature grouped by the group geographic feature
    mode_per_group = df.groupby(group_feature)[target_feature].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    
    # impute missing values 
    missing_mask = df[target_feature].isnull() & df[group_feature].notnull()
    df.loc[missing_mask, target_feature] = df.loc[missing_mask, group_feature].map(mode_per_group)
    
    # drop rows where group feature is missing or if target feature still missing after imputation
    df.dropna(subset=[target_feature, group_feature], inplace=True)

geographical_features = ['fips', 'regionidneighborhood', 'censustractandblock', 'rawcensustractandblock',\
                         'regionidzip', 'regionidcity', 'regionidcounty']
for feature in geographical_features:
    group_feature = 'regionidcounty' if feature != 'regionidcounty' or feature != 'fips' else 'regionidcity'
    impute_geographical_feature(property_df, feature, group_feature)

In [9]:
property_df.shape

(2982285, 34)

In [10]:
# impute categorical feature values
from sklearn.impute import SimpleImputer

cat_imputer = SimpleImputer(strategy='most_frequent') # use mode here as well

categorical_features = ['airconditioningtypeid', 'buildingqualitytypeid', 'heatingorsystemtypeid',
                        'propertycountylandusecode', 'propertylandusetypeid', 'propertyzoningdesc',
                        'regionidcity', 'regionidcounty', 'regionidzip', 'unitcnt',
                        'censustractandblock', 'rawcensustractandblock']

# impute missing values for each feature
for col in categorical_features:
    if col in property_df.columns:
        property_df[[col]] = cat_imputer.fit_transform(property_df[[col]])

/Users/leibmalina/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
# now do the same for the remaining numerical features

num_imputer = SimpleImputer(strategy='median') 

numerical_features = ['basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 
                      'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 
                      'finishedsquarefeet12', 'finishedsquarefeet50', 'garagecarcnt', 'garagetotalsqft', 
                      'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'roomcnt', 
                      'threequarterbathnbr', 'unitcnt', 'yearbuilt', 'numberofstories',
                      'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 
                      'landtaxvaluedollarcnt', 'taxamount', 'fullbathcnt']

for col in numerical_features:
    if col in property_df.columns:
        property_df[[col]] = num_imputer.fit_transform(property_df[[col]])

In [12]:
# confirm all missing values filled
print(property_df.isnull().sum())

parcelid                        0
airconditioningtypeid           0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
heatingorsystemtypeid           0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
propertylandusetypeid           0
propertyzoningdesc              0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidneighborhood            0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
numberofstories                 0
structuretaxvaluedollarcnt      0
taxvaluedollar

### Combining Training Table

In [13]:
# pulling property data into training data
combined_df = train_df.merge(property_df, how='left', on='parcelid')
combined_df.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,14297519,0.025595,2017-01-01,1.0,3.5,4.0,6.0,3.5,3100.0,3100.0,...,0.0,1.0,1998.0,1.0,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13
1,17052889,0.055619,2017-01-01,1.0,1.0,2.0,6.0,1.0,1465.0,1465.0,...,5.0,1.0,1967.0,1.0,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13
2,14186244,0.005383,2017-01-01,1.0,2.0,3.0,6.0,2.0,1243.0,1243.0,...,6.0,1.0,1962.0,1.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13
3,12177905,-0.103410,2017-01-01,1.0,3.0,4.0,8.0,3.0,2376.0,2376.0,...,0.0,1.0,1970.0,1.0,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13
4,10887214,0.006940,2017-01-01,1.0,3.0,3.0,8.0,3.0,1312.0,1312.0,...,0.0,1.0,1964.0,1.0,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13


In [14]:
# remove properties that were removed during data cleaning in the new combined DataFrame
final_property_features = [col for col in property_df.columns if col != 'parcelid']
# remove rows where all property features missing:
missing_property_data = combined_df[final_property_features].isnull().all(axis=1) 
combined_df = combined_df[~missing_property_data]

## Dimensionality Reduction

In [15]:
# convert datetime columns to numerical format (to enable scaling)
for col in combined_df.select_dtypes(include=['datetime']):
    combined_df[f'{col}_year'] = combined_df[col].dt.year
    combined_df[f'{col}_month'] = combined_df[col].dt.month
    combined_df[f'{col}_day'] = combined_df[col].dt.day
    combined_df[f'{col}_weekday'] = combined_df[col].dt.weekday
    combined_df.drop(columns=[col], inplace=True)

In [16]:
# convert categorical to numerical for scaling as well
# using label encoding since one-hot encoding creates 2k+ features, which makes dim. red. infeasible 
categorical_cols = combined_df.select_dtypes(include=['object']).columns

from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    combined_df[col] = le.fit_transform(combined_df[col])
    label_encoders[col] = le  # store to invert encoding later if needed
    
# NOTE: this can create an implied ordinal ordering - can change this if affecting linear model performance

In [17]:
# define levels: increasing number of components
# Baseline: 100% (no dimensionality reduction)
# Level 1: 10% 
# Level 2: 50%
# Level 3: 75%

features = combined_df.drop(columns=['logerror'])
target = combined_df['logerror']

components_10 = int(features.shape[1] * 0.1)
components_50 = int(features.shape[1] * 0.5)
components_75 = int(features.shape[1] * 0.75)

In [18]:
combined_df.columns

Index(['parcelid', 'logerror', 'airconditioningtypeid', 'bathroomcnt',
       'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'unitcnt', 'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'transactiondate_year',
       'transactiondate_month', 'transactiondate_day',
       'transactiondate_weekday'],
      dtype='object')

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [20]:
# key metrics: run time, and data size (memory of dataframe)
# functions to measure these
import time
def measure_runtime(func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    runtime = end_time - start_time
    return result, runtime

def dataframe_memory(df):
    return df.memory_usage(deep=True).sum()

In [21]:
pca_results = {} # store new reduced datasets and metrics

In [22]:
def apply_pca(n_components):
    pca = PCA(n_components=n_components)
    pca_result, runtime = measure_runtime(lambda: pca.fit_transform(features_scaled))
    reduced_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n_components)])
    reduced_df['logerror'] = target
    mem_usage = dataframe_memory(reduced_df)
    return reduced_df, runtime, mem_usage

In [25]:
# Baseline - no reduction
baseline_runtime_start = time.time()
baseline_mem_usage = dataframe_memory(combined_df)
baseline_runtime = time.time() - baseline_runtime_start
pca_results['Baseline'] = (combined_df, baseline_runtime, baseline_mem_usage)

In [27]:
# PCA for 10%, 50%, 75%
from sklearn.decomposition import PCA
for components, label in zip([components_10, components_50, components_75], ['10%', '50%', '75%']):
    reduced_df, runtime, mem_usage = apply_pca(components)
    pca_results[label] = (reduced_df, runtime, mem_usage)

In [28]:
# Output Results
for label, (df, runtime, mem_usage) in pca_results.items():
    print(f"Results for {label}:")
    print(f"Runtime: {runtime:.4f} seconds")
    print(f"Data Size: {mem_usage} bytes")
    print(f"DataFrame head:\n{df.head()}\n")

Results for Baseline:
Runtime: 0.0178 seconds
Data Size: 23584016 bytes
DataFrame head:
   parcelid  logerror  airconditioningtypeid  bathroomcnt  bedroomcnt  \
0  14297519  0.025595                    1.0          3.5         4.0   
1  17052889  0.055619                    1.0          1.0         2.0   
2  14186244  0.005383                    1.0          2.0         3.0   
3  12177905 -0.103410                    1.0          3.0         4.0   
4  10887214  0.006940                    1.0          3.0         3.0   

   buildingqualitytypeid  calculatedbathnbr  calculatedfinishedsquarefeet  \
0                    6.0                3.5                        3100.0   
1                    6.0                1.0                        1465.0   
2                    6.0                2.0                        1243.0   
3                    8.0                3.0                        2376.0   
4                    8.0                3.0                        1312.0   

   finishe